In [143]:
import json
import psycopg2
import time
import pandas as pd
from datetime import datetime
import numpy as np
import math
import pickle
import os
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [144]:
conn = psycopg2.connect(
    port = "5432",
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db"
)

In [145]:
q='''SELECT created_at as created, id as order_iD, total_price as order_amount, email as user_id from orders WHERE
cancelled_at is NULL;
'''
e=pd.read_sql(q,conn)

In [119]:
e=e[e['created']>='2023/01/01']
e=e[e['created']<='2023/12/31']

In [146]:
e

,created,order_id,order_amount,user_id
0,2023-12-08 14:50:03,5341764845706,299.0,agmathew1@gmail.com
1,2023-12-08 14:46:53,5341763338378,224.0,sdfashion19@gmail.com
2,2023-12-08 14:30:36,5341750689930,598.0,shrighadge1965@gmail.com
3,2023-12-08 14:05:10,5341738500234,324.0,rajan.raju58@gmail.com
4,2023-12-08 14:04:08,5341737910410,324.0,shantha7788@gmail.com
...,...,...,...,...
72923,2024-01-08 08:09:56,5383244251274,585.0,deepankarsinghrathore@gmail.com
72924,2024-01-07 23:08:42,5382883868810,720.0,ashankbhatnagar@gmail.com
72925,2024-01-07 23:03:27,5382880723082,655.0,sonakheechaudhury@gmail.com
72926,2024-01-07 21:44:39,5382825181322,199.0,shuvadeepbarua@gmail.com


In [148]:
e['WeekCount']=e['created'].dt.strftime('%Y-w%U')

In [149]:
e=e.sort_values('created')
WeekList=e['WeekCount'].unique()

In [150]:
e["user_id"].nunique()

44505

In [151]:
#change
e["New"] = 0
e['TrxnRank'] = e.groupby('user_id')['created'].rank(method='first')
e.loc[e['TrxnRank']==1,'New'] = 1

In [152]:
e['OrderDate']=e['created'].dt.date
e['OrderDate']=pd.to_datetime(e['OrderDate'])
e['DeltaDaysinOrders']=(e.groupby('user_id').OrderDate.shift() - e.OrderDate).dt.days.abs()
e['DeltaOrderValues']=(e.groupby('user_id').order_amount.shift() - e.order_amount)

In [153]:

x1=e.groupby('user_id').agg(
          LastDate=pd.NamedAgg(column='OrderDate', aggfunc= lambda x: x.nlargest(1).min()),
          FirstDate=pd.NamedAgg(column='OrderDate', aggfunc='min'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          AOV=pd.NamedAgg(column='order_amount', aggfunc='mean'),
          OrderValue=pd.NamedAgg(column='order_amount', aggfunc='sum'),#change
          MeanDaysDelta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          MeanOVDelta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean'),
          TotalTransactions=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          New=pd.NamedAgg(column='New',aggfunc='mean')#change
      ).reset_index()
globalmaxdate=e['OrderDate'].max()
x1['Recency']=globalmaxdate-x1['LastDate']
x1['Recency']=x1['Recency'].dt.days.abs()
x1['ActivatedQuater']=x1['FirstDate'].dt.to_period('Q')
x1['MaturityAge']=np.nan
x1.loc[x1['Frequency']>1,"MaturityAge"]=(x1['LastDate']-x1['FirstDate']).dt.days.abs()
x1['MeanDaysDelta']=x1['MeanDaysDelta'].fillna(0)
x1['MeanOVDelta']=x1['MeanOVDelta'].fillna(0)

r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
x1['r_score'] = x1['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
x1['f2d_score'] = x1['Frequency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
x1['AOV_score'] = x1['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
x1['m_score']=0
x1.loc[x1['MaturityAge'].notnull(),'m_score']= x1.loc[x1['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
x1['DelAOV'] = x1['MeanOVDelta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
x1['SumScore'] = x1['r_score'] + x1['f2d_score'] + x1['m_score'] + x1['AOV_score']+ x1['DelAOV']

def CohortLables_3(df):
    if ((df['Frequency']==1) & (df['r_score']>2)):
        return "New"
    elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
        return 'Churned (Single Use Only)'
    elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
        return 'Recent & Loyals'
    elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 2)):
        return 'Champions'
    else:
        return 'Churned (Repeat Use)'
x1['CohortLabel'] = x1.apply(CohortLables_3, axis=1)

In [174]:
# change
segstats=x1.groupby(
  'CohortLabel'
).agg(
      Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
      Recency=pd.NamedAgg(column='Recency', aggfunc='mean'),
      Frequency=pd.NamedAgg(column='Frequency', aggfunc='mean'),
      OrderValue=pd.NamedAgg(column='OrderValue', aggfunc='sum'), #change
      #MeanDaysDelta=pd.NamedAgg(column='MeanDaysDelta', aggfunc='mean'),
      #MeanOVDelta=pd.NamedAgg(column='MeanOVDelta', aggfunc='mean'),
      #FrequencyperDeltaDays=pd.NamedAgg(column='FrequencyperDeltaDays', aggfunc='mean'),
    Maturity=pd.NamedAgg(column='MaturityAge', aggfunc='mean'),
    OrderCount=pd.NamedAgg(column='Frequency', aggfunc='sum'),
).reset_index()
segstats["AOV"]=segstats['OrderValue']/segstats['OrderCount']
segstats['OrderValueMix']=segstats['OrderValue']/segstats['OrderValue'].sum()
segstats['UsersMix']=segstats['Users']/segstats['Users'].sum()
segstats['AOV_Contribution']=segstats['OrderValue']/segstats['OrderCount'].sum()
segstats['OrderCountMix']=segstats['OrderCount']/segstats['OrderCount'].sum()

In [175]:
# change
overall_totals = segstats.drop(columns='CohortLabel').sum().tolist()
overall_totals.insert(0, 'Overall')
overall_df = pd.DataFrame([overall_totals], columns=segstats.columns)
segstats = pd.concat([segstats, overall_df], ignore_index=True)

In [176]:
segstats

,CohortLabel,Users,Recency,Frequency,OrderValue,Maturity,OrderCount,AOV,OrderValueMix,UsersMix,AOV_Contribution,OrderCountMix
0,Champions,2715.0,108.370166,4.293923,9874493.95,325.601842,11658.0,847.014406,0.216679,0.061004,135.400586,0.159856
1,Churned (Repeat Use),3525.0,523.738440,5.613901,12867180.56,159.890780,19789.0,650.218837,0.282349,0.079205,176.436767,0.271350
2,Churned (Single Use Only),18727.0,562.114487,1.000000,13529128.12,NaN,18727.0,722.439692,0.296874,0.420784,185.513494,0.256788
3,New,17047.0,117.002581,1.000000,6737763.20,NaN,17047.0,395.246272,0.147849,0.383036,92.389250,0.233751
4,Recent & Loyals,2491.0,98.960658,2.291048,2563330.60,24.067443,5707.0,449.155528,0.056248,0.055971,35.148785,0.078255
5,Overall,44505.0,1410.186332,14.198871,45571896.43,509.560065,72928.0,3064.074736,1.000000,1.000000,624.888883,1.000000


In [177]:
# change
def weighted_average_column(df, column_name, weights_column):
    data = df[column_name]
    weights = df[weights_column]

    if weights.sum() == 0:
        raise ValueError("Total weight must be greater than 0.")

    return (data * weights).sum() / weights.sum()

h = weighted_average_column(segstats, 'Maturity', 'Users')

In [178]:
#change
segstats.loc[segstats['CohortLabel'] == 'Overall', 'Recency'] = segstats['Recency'].min()
segstats.loc[segstats['CohortLabel'] == 'Overall', 'Frequency'] = segstats['OrderCount'].sum()/segstats['Users'].sum()
segstats.loc[segstats['CohortLabel'] == 'Overall', 'AOV'] = segstats['OrderValue'].sum()/segstats['OrderCount'].sum()
segstats.loc[segstats['CohortLabel'] == 'Overall', 'Maturity']= h

In [179]:
# change
segstats = (
    segstats
    .fillna(-999999)
    .astype({'Users': int, 'Recency': int, 'Frequency': int, 'Maturity': int, 'OrderCount': int})
    .round({'OrderValue': 2, 'AOV': 2, 'AOV_Contribution': 2, 'OrderValueMix': 2, 'UsersMix': 2, 'OrderCountMix': 2})
    .replace(-999999, np.nan)
)

In [180]:
segstats

,CohortLabel,Users,Recency,Frequency,OrderValue,Maturity,OrderCount,AOV,OrderValueMix,UsersMix,AOV_Contribution,OrderCountMix
0,Champions,2715,108,4,9874493.95,325.0,11658,847.01,0.22,0.06,135.40,0.16
1,Churned (Repeat Use),3525,523,5,12867180.56,159.0,19789,650.22,0.28,0.08,176.44,0.27
2,Churned (Single Use Only),18727,562,1,13529128.12,NaN,18727,722.44,0.30,0.42,185.51,0.26
3,New,17047,117,1,6737763.20,NaN,17047,395.25,0.15,0.38,92.39,0.23
4,Recent & Loyals,2491,98,2,2563330.60,24.0,5707,449.16,0.06,0.06,35.15,0.08
5,Overall,44505,98,1,45571896.43,271.0,72928,624.89,1.00,1.00,624.89,1.00


In [181]:
# ocm_percent_lst = []
# for i in range(len(segstats)):
#     num = segstats["OrderCountMix"].iloc[i]
#     percentage = "{:.0%}".format(num)
#     ocm_percent_lst.append(percentage)
# segstats["OrderCountMixPercentage"] = ocm_percent_lst
segstats["OrderCountMix"] = segstats["OrderCountMix"].apply(lambda x: "{:.0%}".format(x))
segstats["OrderValueMix"] = segstats["OrderValueMix"].apply(lambda x: "{:.0%}".format(x))
segstats["UsersMix"] = segstats["UsersMix"].apply(lambda x: "{:.0%}".format(x))

In [182]:
segstats

,CohortLabel,Users,Recency,Frequency,OrderValue,Maturity,OrderCount,AOV,OrderValueMix,UsersMix,AOV_Contribution,OrderCountMix
0,Champions,2715,108,4,9874493.95,325.0,11658,847.01,22%,6%,135.40,16%
1,Churned (Repeat Use),3525,523,5,12867180.56,159.0,19789,650.22,28%,8%,176.44,27%
2,Churned (Single Use Only),18727,562,1,13529128.12,NaN,18727,722.44,30%,42%,185.51,26%
3,New,17047,117,1,6737763.20,NaN,17047,395.25,15%,38%,92.39,23%
4,Recent & Loyals,2491,98,2,2563330.60,24.0,5707,449.16,6%,6%,35.15,8%
5,Overall,44505,98,1,45571896.43,271.0,72928,624.89,100%,100%,624.89,100%


In [135]:
ovm_percent_lst = []
for i in range(len(segstats)):
    num = segstats["OrderValueMix"].iloc[i]
    percentage = "{:.0%}".format(num)
    ovm_percent_lst.append(percentage)
segstats["OrderValueMixPercentage"] = ovm_percent_lst

In [136]:
segstats

,CohortLabel,Users,Recency,Frequency,OrderValue,Maturity,OrderCount,AOV,OrderValueMix,UsersMix,AOV_Contribution,OrderCountMix,OrderCountMixPercentage,OrderValueMixPercentage
0,Champions,1797,56,3,5057950.35,173.0,6486,779.83,0.24,0.06,133.15,0.17,17%,24%
1,Churned (Repeat Use),1726,215,2,3461817.28,59.0,4329,799.68,0.16,0.06,91.13,0.11,11%,16%
2,Churned (Single Use Only),12579,243,1,7633317.05,NaN,12579,606.83,0.36,0.44,200.95,0.33,33%,36%
3,New,10875,69,1,3811720.55,NaN,10875,350.50,0.18,0.38,100.34,0.29,29%,18%
4,Recent & Loyals,1633,63,2,1379780.26,17.0,3718,371.11,0.06,0.06,36.32,0.10,10%,6%
5,Overall,28610,56,1,21344585.49,132.0,37987,561.89,1.00,1.00,561.89,1.00,100%,100%


In [137]:
segstats_graph = segstats.drop(segstats.index[-1])
column_to_plot = 'OrderCount'
x_labels = segstats_graph.index
plt.figure(figsize=(12, 6))  # Adjust the figure size if needed
plt.bar(x_labels, segstats_graph[column_to_plot])
plt.xlabel('CohortLabel')
plt.ylabel(column_to_plot)
plt.title(f'{column_to_plot} across Customer Cohorts')
plt.xticks(fontsize=8)
plt.show()

In [138]:
# def CohortLables_1(df):
#   if (df['New']==1 and df['AOV_score']>2):
#     return 'New&High Value'
#   elif (df['New']==1 and df['AOV_score']<=2):
#     return 'New&Low Value'
#   elif (df['New']<1 and df['AOV_score']<=2):
#     return 'Returning&Low Value'
#   else:
#     return 'Returning&High Value'

# def CohortLables_2(df):
#   if ((df['SumScore'] >= 18) and (df['Frequency']>1)):
#       return 'Champions'
#   elif ((df['m_score'] <=1) and (df['SumScore'] >= 10)and (df['SumScore'] < 18) and (df['r_score'] >1)):
#       return 'New&Potential'
#   elif ((df['m_score'] >1) and (df['SumScore'] >= 14)and (df['SumScore'] < 18)and (df['r_score'] >1)):
#       return 'Mature&Loyal'
#   elif ((df['m_score'] >1) and (df['SumScore'] >= 10)and (df['SumScore'] < 14)and (df['r_score'] >1)):
#       return 'Mature&Slow'
#   elif ((df['m_score'] <=1) and (df['SumScore'] >= 5)and (df['SumScore'] < 10)and (df['r_score'] >1)):
#       return 'New&Slow'
#   elif ((df['m_score'] >1) and (df['SumScore'] >= 5)and (df['SumScore'] < 10)and (df['r_score'] >1)):
#       return 'Mature&ChurnRisk'
#   else:
#       return 'Churned'
# if (count_customer<500 or no_of_days.days < 180):
#   x1['CohortLabel'] = x1.apply(CohortLables_1, axis=1)
# else:
#   x1['CohortLabel'] = x1.apply(CohortLables_2, axis=1)

In [139]:
# count_customer = e['user_id'].nunique()
# maxDate = e.OrderDate.max()
# minDate = e.OrderDate.min()
# no_of_days = maxDate - minDate
# no_of_days.days